In [1]:
from enum import Enum
import requests
import pandas as pd
import os

In [2]:
class KialoSort(Enum):
  RANK_ACTIVITY = "rank_and_latest_activity"
  VIEW          = "view_count"
  LAST_ACTIVITY = "latest_activity"

class KialoFilter(Enum):
  PROMOTED      = "promoted"
  PARTICIPATE   = "participate"
  LAST_ACTIVITY = "latest_activity"
  TAG           = "tag"
  TAG_ALL       = "tag_all"

In [3]:
class KialoTool:
  def getTags(self):
    tags = requests.get("https://www.kialo.com/api/v1/discussiontags")
    return [item['tagName'] for item in tags.json()["tags"]]

  def getDiscussions(self, filter: KialoFilter, sort: KialoSort, limit=3000):
    req = "https://www.kialo.com/api/v1/discussions?filter=" + str(filter.value) + "&sort=" + str(sort.value) + "&limit=" + str(limit) + "&skip=0"
    tags = requests.get(req)
    return tags.json()["discussions"]

  def discussions2urlID(self, discussions, export=False):
    idsUrl = [x["title"].lower().replace("?", "").replace(" ", "-")+"-"+str(x["id"]) for x in discussions]
    tags = [x["tags"] for x in discussions]
    if export:
      pd.DataFrame.from_dict({"kialoUrlId" : idsUrl, "tags" : tags}).to_csv("../rawData/kialo/kialo-url-ids.csv")

    return idsUrl

k = KialoTool()

### Export most active and high ranked kialo discussions

In [4]:
k.discussions2urlID(k.getDiscussions(filter=KialoFilter.TAG, sort=KialoSort.RANK_ACTIVITY), export=True)

['are-purity-pledges-harmful-29355',
 'should-american-football-be-banned-10143',
 'should-there-be-one-world-state-30339',
 'do-fair-trade-products-cause-more-good-than-harm-31027',
 'should-failing-banks-receive-bailouts-from-the-government-64072',
 'should-governments-ever-limit-free-speech-22454',
 'is-the-scientific-process-objective-17768',
 'time-for-a-maximum-wage-should-the-us-limit-ceo-pay-2141',
 'should-quotas-for-women-on-boards-and-in-managerial-positions-be-mandatory-9821',
 'should-election-campaigns-only-be-funded-by-the-government-26876',
 'should-people-marry-outside-their-religious-faith-30171',
 'what-are-the-benefits-/-harms-of-critical-race-theory-40007',
 'should-fake-news-be-illegal-35818',
 'should-high-income-countries-take-in-refugees-2214',
 "should-everyone's-wealth-and-income-information-be-publicly-available-18009",
 'should-teachers-be-allowed-to-wear-religious-symbols-at-school-12626',
 'should-begging-for-money-be-illegal-16454',
 'compulsory-voting:-

In [5]:
kialoUsername = "filler"
secret      = "filler"

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

def downloadDiscussions(disscusionUrlIds):
  path = os.path.abspath("../rawData/kialo/debates")


  prefs = {"download.default_directory": path }
  options = Options()
  options.add_experimental_option("prefs", prefs)
  driver = webdriver.Chrome(options = options)


  driver.get("https://www.kialo.com/login")

  #Login
  id          = driver.find_element(By.ID, "emailOrUsername")
  password    = driver.find_element(By.ID, "password")
  loginButton = driver.find_element(By.CLASS_NAME, "login-form__submit")
  id.send_keys(kialoUsername)
  password.send_keys(secret)

  loginButton.click()

  myElem = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'home-page-section__header')))


  for urlId in tqdm(disscusionUrlIds):
    driver.get("https://www.kialo.com/export/" + urlId + ".txt")
    time.sleep(1.5)


In [7]:
downloadDiscussions(k.discussions2urlID(k.getDiscussions(filter=KialoFilter.TAG, sort=KialoSort.RANK_ACTIVITY), export=True))

100%|██████████| 1000/1000 [29:04<00:00,  1.74s/it]
